In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [20]:
df = pd.read_csv("Datasets/IEAGlobalEV.csv")
df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011.0,Vehicles,4.900000e+01
1,Australia,Historical,EV sales share,Cars,EV,2011.0,percent,6.500000e-03
2,Australia,Historical,EV stock share,Cars,EV,2011.0,percent,4.600000e-04
3,Australia,Historical,EV sales,Cars,BEV,2011.0,Vehicles,4.900000e+01
4,Australia,Historical,EV sales,Cars,BEV,2012.0,Vehicles,1.700000e+02
...,...,...,...,...,...,...,...,...
9537,World,Projection-APS,EV stock,Trucks,PHEV,2030.0,Vehicles,9.100000e+05
9538,World,Projection-APS,EV stock,Vans,BEV,2030.0,Vehicles,1.200000e+07
9539,World,Projection-APS,EV stock,Vans,PHEV,2030.0,Vehicles,1.300000e+06
9540,World,Projection-APS,EV stock,Cars,BEV,2030.0,Vehicles,1.800000e+08


Lets check how much the datat needs to be cleaned


In [21]:
df = df.drop_duplicates()
df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011.0,Vehicles,4.900000e+01
1,Australia,Historical,EV sales share,Cars,EV,2011.0,percent,6.500000e-03
2,Australia,Historical,EV stock share,Cars,EV,2011.0,percent,4.600000e-04
3,Australia,Historical,EV sales,Cars,BEV,2011.0,Vehicles,4.900000e+01
4,Australia,Historical,EV sales,Cars,BEV,2012.0,Vehicles,1.700000e+02
...,...,...,...,...,...,...,...,...
9537,World,Projection-APS,EV stock,Trucks,PHEV,2030.0,Vehicles,9.100000e+05
9538,World,Projection-APS,EV stock,Vans,BEV,2030.0,Vehicles,1.200000e+07
9539,World,Projection-APS,EV stock,Vans,PHEV,2030.0,Vehicles,1.300000e+06
9540,World,Projection-APS,EV stock,Cars,BEV,2030.0,Vehicles,1.800000e+08


In [22]:
#lets check the datatype in the dataframe
print(df.dtypes)

#lets convert each colum that has object into strings so its easier to work with
df = df.apply(lambda col: col.astype(str) if col.dtype == 'object' else col)

region         object
category       object
parameter      object
mode           object
powertrain     object
year          float64
unit           object
value         float64
dtype: object


## Looks like it does not need much more cleaning. Let's first of all remove the rows where region is world (since we only care about each country separtely), where parameter is not EV sales (because there are EV stocks and such parameters) and where unit is percent because we only want absolute values

In [23]:
df = df[df["region"].str.contains("World") == False]
df = df[df["region"].str.contains("world") == False]
df = df[df["parameter"].str.contains("EV sales") == True]
df = df[df["unit"].str.contains("percent") == False]
df

,region,category,parameter,mode,powertrain,year,unit,value
3,Australia,Historical,EV sales,Cars,BEV,2011.0,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012.0,Vehicles,170.0
9,Australia,Historical,EV sales,Cars,PHEV,2012.0,Vehicles,80.0
10,Australia,Historical,EV sales,Cars,PHEV,2013.0,Vehicles,100.0
15,Australia,Historical,EV sales,Cars,BEV,2013.0,Vehicles,190.0
...,...,...,...,...,...,...,...,...
8701,USA,Projection-STEPS,EV sales,Trucks,PHEV,2030.0,Vehicles,34000.0
8702,USA,Projection-STEPS,EV sales,Vans,BEV,2030.0,Vehicles,370000.0
8703,USA,Projection-STEPS,EV sales,Vans,PHEV,2030.0,Vehicles,31000.0
8704,USA,Projection-STEPS,EV sales,Cars,BEV,2030.0,Vehicles,6800000.0


In [24]:
#to see if our prediction is accurate we should get the actual ammount produced in 2025
total_production_2030 = df[df['year'] == 2030]['value'].sum()
# print(f"Total EV production for 2030: {total_production_2025}")

print(f"Total EV production for 2030: {total_production_2030 / 1_000_000:.2f}M")

Total EV production for 2030: 72.31M


### Also lets cremove the category that has projection in the name and where the parameter is not just EV Sales

In [25]:
df = df[df["category"].str.contains("Historical") == True]
df = df[df["parameter"].str.contains("EV sales") == True]
df

,region,category,parameter,mode,powertrain,year,unit,value
3,Australia,Historical,EV sales,Cars,BEV,2011.0,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012.0,Vehicles,170.0
9,Australia,Historical,EV sales,Cars,PHEV,2012.0,Vehicles,80.0
10,Australia,Historical,EV sales,Cars,PHEV,2013.0,Vehicles,100.0
15,Australia,Historical,EV sales,Cars,BEV,2013.0,Vehicles,190.0
...,...,...,...,...,...,...,...,...
8492,USA,Historical,EV sales,Cars,BEV,2020.0,Vehicles,230000.0
8527,USA,Historical,EV sales,Cars,BEV,2021.0,Vehicles,470000.0
8539,USA,Historical,EV sales,Cars,PHEV,2021.0,Vehicles,160000.0
8542,USA,Historical,EV sales,Cars,PHEV,2022.0,Vehicles,190000.0


### In tableu we just want to view historical data for the EV sales for a specific country! above we can even remove two columns; category and Parameter. Because they are always the same!


In [26]:
df.drop(['category', 'parameter', 'powertrain'], axis=1, inplace=True)
df

,region,mode,year,unit,value
3,Australia,Cars,2011.0,Vehicles,49.0
4,Australia,Cars,2012.0,Vehicles,170.0
9,Australia,Cars,2012.0,Vehicles,80.0
10,Australia,Cars,2013.0,Vehicles,100.0
15,Australia,Cars,2013.0,Vehicles,190.0
...,...,...,...,...,...
8492,USA,Cars,2020.0,Vehicles,230000.0
8527,USA,Cars,2021.0,Vehicles,470000.0
8539,USA,Cars,2021.0,Vehicles,160000.0
8542,USA,Cars,2022.0,Vehicles,190000.0


In [27]:
#as i can see, Australia for the cars in 2011 with uni vehicles has two different values. This is because of a column
#that was removed that had the type of EV. i do not wish do discriminate based on the EV type so i will sum everything

df = df.groupby(['region', 'mode', 'year', 'unit'], as_index=False)['value'].sum()
df

,region,mode,year,unit,value
0,Australia,Cars,2011.0,Vehicles,49.0
1,Australia,Cars,2012.0,Vehicles,250.0
2,Australia,Cars,2013.0,Vehicles,290.0
3,Australia,Cars,2014.0,Vehicles,1320.0
4,Australia,Cars,2015.0,Vehicles,1760.0
...,...,...,...,...,...
1122,United Kingdom,Vans,2018.0,Vehicles,1650.0
1123,United Kingdom,Vans,2019.0,Vehicles,3710.0
1124,United Kingdom,Vans,2020.0,Vehicles,6260.0
1125,United Kingdom,Vans,2021.0,Vehicles,13200.0


In [28]:
#Now finally, the columns year and value are floats since they have the .0 however, they are always integers so i will convert them
df['year'] = df['year'].astype(int)
df['value'] = df['value'].astype(int)

df

,region,mode,year,unit,value
0,Australia,Cars,2011,Vehicles,49
1,Australia,Cars,2012,Vehicles,250
2,Australia,Cars,2013,Vehicles,290
3,Australia,Cars,2014,Vehicles,1320
4,Australia,Cars,2015,Vehicles,1760
...,...,...,...,...,...
1122,United Kingdom,Vans,2018,Vehicles,1650
1123,United Kingdom,Vans,2019,Vehicles,3710
1124,United Kingdom,Vans,2020,Vehicles,6260
1125,United Kingdom,Vans,2021,Vehicles,13200


### We need another dataset with the population figures of each country by year so we can add them to the original DF and get the  value per capita 

In [29]:
#reading the file
dfPop = pd.read_csv("Datasets/population-and-demography.csv")
dfPop

#only want the years between
df_filtered = dfPop[(dfPop['Year'] >= 2011) & (dfPop['Year'] <= 2022)]

#i only need these 3 columns
df_filtered = df_filtered[['Country name', 'Year', 'Population']]


In [30]:
#Now join both dataframes!

df_merged = pd.merge(df, df_filtered, left_on=['region', 'year'], right_on=['Country name', 'Year'])
df_merged.drop(['Country name', 'Year'], axis=1, inplace=True)
df_merged


,region,mode,year,unit,value,Population
0,Australia,Cars,2011,Vehicles,49,22357032
1,Australia,Cars,2012,Vehicles,250,22729272
2,Australia,Cars,2013,Vehicles,290,23111788
3,Australia,Cars,2014,Vehicles,1320,23469578
4,Australia,Cars,2015,Vehicles,1760,23820240
...,...,...,...,...,...,...
784,United Kingdom,Vans,2020,Vehicles,6260,67059470
785,United Kingdom,Buses,2021,Vehicles,540,67281040
786,United Kingdom,Cars,2021,Vehicles,310000,67281040
787,United Kingdom,Trucks,2021,Vehicles,150,67281040


In [31]:
#Finally we add a column with the per capita value!
df_merged['Per Capita Value'] = df_merged['value'] / df_merged['Population']
df_merged

#very low values. lets see the demand NOT PER CAPITA, but per 10 000 people!
df_merged['value_per_10_000'] = df_merged['Per Capita Value'] * 10000
df_merged

,region,mode,year,unit,value,Population,Per Capita Value,value_per_10_000
0,Australia,Cars,2011,Vehicles,49,22357032,0.000002,0.021917
1,Australia,Cars,2012,Vehicles,250,22729272,0.000011,0.109990
2,Australia,Cars,2013,Vehicles,290,23111788,0.000013,0.125477
3,Australia,Cars,2014,Vehicles,1320,23469578,0.000056,0.562430
4,Australia,Cars,2015,Vehicles,1760,23820240,0.000074,0.738867
...,...,...,...,...,...,...,...,...
784,United Kingdom,Vans,2020,Vehicles,6260,67059470,0.000093,0.933500
785,United Kingdom,Buses,2021,Vehicles,540,67281040,0.000008,0.080260
786,United Kingdom,Cars,2021,Vehicles,310000,67281040,0.004608,46.075388
787,United Kingdom,Trucks,2021,Vehicles,150,67281040,0.000002,0.022295


In [32]:
#way too much decimals, lets round to the 4
df_merged['value_per_10_000'] = df_merged['value_per_10_000'].round(4)
df_merged

,region,mode,year,unit,value,Population,Per Capita Value,value_per_10_000
0,Australia,Cars,2011,Vehicles,49,22357032,0.000002,0.0219
1,Australia,Cars,2012,Vehicles,250,22729272,0.000011,0.1100
2,Australia,Cars,2013,Vehicles,290,23111788,0.000013,0.1255
3,Australia,Cars,2014,Vehicles,1320,23469578,0.000056,0.5624
4,Australia,Cars,2015,Vehicles,1760,23820240,0.000074,0.7389
...,...,...,...,...,...,...,...,...
784,United Kingdom,Vans,2020,Vehicles,6260,67059470,0.000093,0.9335
785,United Kingdom,Buses,2021,Vehicles,540,67281040,0.000008,0.0803
786,United Kingdom,Cars,2021,Vehicles,310000,67281040,0.004608,46.0754
787,United Kingdom,Trucks,2021,Vehicles,150,67281040,0.000002,0.0223


### Now we can export this to a csv file and Visualize it in Tableau!

In [34]:
# df_merged.to_csv('DemandForEVsByCountryUntil2022.csv')
df_merged.to_csv('DemandForEVsByCountryUntil2022.csv', sep=';', float_format='%.10f', index=False)